# Task-Interface

Die *Task*-Interface zum Cluster präsentiert die *Engines* als fehlertolerantes, dynamisches LoadBalancing für von *Workers*. Im Gegensatz zur *Direct*-Interface gibt es bei der *Task*-Interface keinen direkten Zugriff auf einzelne *Engines*. Indem der IPython-Scheduler die *Worker* zuzuweist, wird die Schnittstelle einfacher und zugleich leistungsfähiger.

Das Beste ist jedoch, dass beide Schnittstellen gleichzeitig verwendet werden können, um die jeweiligen Stärken zu nutzen. Wenn Berechnungen nicht von früheren Ergebnissen abhängen, ist, ist das *Task*-Interface ideal:

## Erstellen einer `LoadBalancedView`-Instanz



In [1]:
import ipyparallel as ipp

In [2]:
rc = ipp.Client()

In [3]:
rc = ipp.Client(url_file='/Users/veit/.ipython/profile_mpi/security/ipcontroller-client.json')

In [4]:
rc = ipp.Client(profile='mpi')

In [5]:
lview = rc.load_balanced_view()

`load_balanced_view` ist die Standardansicht.

> **Siehe auch**: [Views](https://ipyparallel.readthedocs.io/en/latest/details.html#parallel-details).

## Schnelle und einfache Parallelität



### `map()`-LoadBalancedView

In [6]:
lview.block = True
serial_result = map(lambda x:x**10, range(32))
parallel_result = lview.map(lambda x:x**10, range(32))
serial_result==parallel_result

True

### `@lview.parallel()`-Decorator

In [7]:
@lview.parallel()
def f(x):
    return 10.0*x**4

f.map(range(32))

[0.0,10.0,160.0,…]

## Abhängigkeiten

> **Hinweis**: Beachtet bitte, dass der reine ZeroMQ-Scheduler keine Abhängigkeiten unterstützt.

### Funktionsabhängigkeiten

`UnmetDependency`

#### `@ipp.require`-Decorator

#### `@ipp.depend`-Decorator

#### `dependent`-Objekt



### `Dependency`

In [ ]:
client.block=False

ar = lview.apply(f, args, kwargs)
ar2 = lview.apply(f2)

with lview.temp_flags(after=[ar,ar2]):
    ar3 = lview.apply(f3)

with lview.temp_flags(follow=[ar], timeout=2.5)
    ar4 = lview.apply(f3)

> **Siehe auch:** Manche parallele Workloads können als [Directed acyclic graph](https://en.wikipedia.org/wiki/Directed_acyclic_graph) (DAG) beschrieben werden. In [DAG Dependencies](https://ipyparallel.readthedocs.io/en/latest/dag_dependencies.html#dag-dependencies) wird anhand eines Beispiels beschrieben, wie [NetworkX](../../jupyter/ipywidgets/networkx.html) zur Darstellung der Task-Abhängigkeiten als DAGs dargestellt werden.

### `ImpossibleDependency`

`retries` und `resubmit`

## Schedulers

In [ ]:
ipcontroller --scheme=lru

| Schema        | Beschreibung |
| ------------- | ------------ |
| `lru`         | ***Least Recently Used***: Weist die Worker immer der zuletzt verwendeten *Engine* zu. Ähnlich *Round-Robin* berücksichtigt es jedoch nicht die Laufzeit jeder einzelnen Aufgabe. |
| `plainrandom` | ***Plain Random***: Wählt zufällig die *Engine* aus, die ausgeführt werden soll. |
| `twobin`      | ***Two-Bin Random***: Benötigt `numpy`. Wählt zwei *Engines* zufällig aus und verwendt die `lru` der beiden. Dies ist häufig besser als die rein zufällige Verteilung, erfordert jedoch einen höheren Rechenaufwand. |
| `leastload`   | ***Least Load***: Standardschema, das die *Engine* immer Aufgaben mit den wenigsten ausstehenden Aufgaben zuweist. |
| `weighted`    | ***Weighted Two-Bin Random***: Gewichtetes ***Two-Bin Random***-Schema. |
